# 아파트 정책/정세 에이전트 (Azure OpenAI + 로컬 PDF 지식)

- 주요 지식원: `DATA/real_estate_policy_info.pdf`, `DATA/real_estate_news.pdf`
- 목적: PDF에 포함된 정책/정세 정보를 근거로 질문에 답변


In [8]:
import json
import os
import re
import subprocess
import sys
from pathlib import Path

from dotenv import load_dotenv

# .env를 cwd부터 상위 폴더로 탐색 로드
_cwd = Path.cwd().resolve()
for _ in range(6):
    _env = _cwd / '.env'
    if _env.exists():
        load_dotenv(_env)
        break
    if _cwd.parent == _cwd:
        break
    _cwd = _cwd.parent
else:
    load_dotenv()

# pypdf가 없으면 현재 커널 환경에 설치
try:
    from pypdf import PdfReader
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pypdf'])
    from pypdf import PdfReader

from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential


def _try_apply_azure_env_from_budget_notebook(path='budget_agent.ipynb'):
    """budget_agent.ipynb에 하드코딩된 Azure 설정이 있으면 현재 환경변수에 반영"""
    p = Path(path)
    if not p.exists():
        return False

    try:
        nb = json.loads(p.read_text(encoding='utf-8'))
    except Exception:
        return False

    endpoint = None
    deployment = None
    for c in nb.get('cells', []):
        if c.get('cell_type') != 'code':
            continue
        src = ''.join(c.get('source', []))
        m1 = re.search(r'os\.environ\["AZURE_OPENAI_ENDPOINT"\]\s*=\s*"([^"]+)"', src)
        m2 = re.search(r'os\.environ\["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"\]\s*=\s*"([^"]+)"', src)
        if m1 and not endpoint:
            endpoint = m1.group(1).strip()
        if m2 and not deployment:
            deployment = m2.group(1).strip()

    if endpoint and not os.getenv('AZURE_OPENAI_ENDPOINT'):
        os.environ['AZURE_OPENAI_ENDPOINT'] = endpoint
    if deployment and not os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'):
        os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'] = deployment

    return bool(endpoint or deployment)


_applied = _try_apply_azure_env_from_budget_notebook()

print('✅ 라이브러리/환경 로드 완료')
print('   budget_agent 설정 반영:', '예' if _applied else '아니오')
print('   AZURE_OPENAI_ENDPOINT:', '설정됨' if os.getenv('AZURE_OPENAI_ENDPOINT') else '미설정')
print('   AZURE_OPENAI_CHAT_DEPLOYMENT_NAME:', '설정됨' if os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME') else '미설정')


✅ 라이브러리/환경 로드 완료
   budget_agent 설정 반영: 예
   AZURE_OPENAI_ENDPOINT: 설정됨
   AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: 설정됨


In [9]:
class LocalRealEstatePdfTool:
    """로컬 PDF를 로딩해 정책/정세 질의에 근거를 제공하는 도구"""

    def __init__(self, data_dir='DATA'):
        self.data_dir = Path(data_dir)
        self.pdf_candidates = [
            'real_estate_policy_info.pdf',
            'real_estate_news.pdf',
            'reak_estate_news.pdf',
        ]
        self.chunks = []
        self._loaded = False

    def _resolve_pdf_paths(self):
        paths = []
        for name in self.pdf_candidates:
            p = self.data_dir / name
            if p.exists():
                paths.append(p)
        return paths

    def _split_text(self, text: str, chunk_size: int = 900) -> list[str]:
        text = re.sub(r'\s+', ' ', (text or '')).strip()
        if not text:
            return []
        chunks = []
        start = 0
        while start < len(text):
            end = min(start + chunk_size, len(text))
            chunks.append(text[start:end])
            start = end
        return chunks

    def load_local_pdfs(self) -> str:
        """로컬 PDF를 읽어 텍스트 청크를 메모리에 적재"""
        paths = self._resolve_pdf_paths()
        self.chunks = []

        for pdf_path in paths:
            try:
                reader = PdfReader(str(pdf_path))
                for page_idx, page in enumerate(reader.pages, start=1):
                    page_text = page.extract_text() or ''
                    for c in self._split_text(page_text):
                        self.chunks.append({
                            'source_file': pdf_path.name,
                            'page': page_idx,
                            'text': c,
                        })
            except Exception:
                continue

        self._loaded = True
        return json.dumps({
            'loaded_files': [p.name for p in paths],
            'chunk_count': len(self.chunks),
            'status': 'ok' if self.chunks else 'empty'
        }, ensure_ascii=False)

    def answer_from_local_pdfs(self, question: str, top_k: int = 5) -> str:
        """질문과 관련된 PDF 근거 조각을 반환"""
        if (not self._loaded) or (not self.chunks):
            self.load_local_pdfs()

        if not self.chunks:
            return json.dumps({
                'question': question,
                'evidence': [],
                'note': 'PDF 로딩 실패 또는 텍스트 추출 결과 없음'
            }, ensure_ascii=False)

        tokens = re.findall(r'[가-힣A-Za-z0-9]{2,}', (question or '').lower())
        if not tokens:
            tokens = ['정책']

        scored = []
        for chunk in self.chunks:
            corpus = chunk['text'].lower()
            score = sum(corpus.count(t) for t in tokens)
            if score > 0:
                scored.append((score, chunk))

        if not scored:
            scored = [(0, c) for c in self.chunks[:top_k]]

        scored.sort(key=lambda x: x[0], reverse=True)
        evidence = []
        for score, c in scored[:top_k]:
            evidence.append({
                'score': score,
                'source_file': c['source_file'],
                'page': c['page'],
                'snippet': c['text'][:450],
            })

        return json.dumps({
            'question': question,
            'evidence': evidence,
            'loaded_chunk_count': len(self.chunks),
        }, ensure_ascii=False)

print('✅ LocalRealEstatePdfTool 정의 완료')


✅ LocalRealEstatePdfTool 정의 완료


In [10]:
def create_apt_policy_agent(credential=None, endpoint=None, deployment_name=None):
    tool = LocalRealEstatePdfTool(data_dir='DATA')
    tool.load_local_pdfs()

    # 호출 인자 > 환경변수 순으로 설정
    resolved_endpoint = endpoint or os.getenv('AZURE_OPENAI_ENDPOINT')
    resolved_deployment = (
        deployment_name
        or os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')
        or os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
        or os.getenv('AZURE_OPENAI_DEPLOYMENT')
    )

    if resolved_endpoint:
        os.environ['AZURE_OPENAI_ENDPOINT'] = resolved_endpoint
    if resolved_deployment:
        os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'] = resolved_deployment

    if not os.getenv('AZURE_OPENAI_ENDPOINT'):
        raise ValueError("AZURE_OPENAI_ENDPOINT 가 필요합니다. budget_agent.ipynb처럼 환경변수 또는 함수 인자로 설정하세요.")
    if not os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'):
        raise ValueError("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME 이 필요합니다. budget_agent.ipynb처럼 환경변수 또는 함수 인자로 설정하세요.")

    use_api_key = bool(os.getenv('AZURE_OPENAI_API_KEY'))
    if use_api_key:
        chat_client = AzureOpenAIChatClient(api_key=os.environ['AZURE_OPENAI_API_KEY'])
    else:
        chat_client = AzureOpenAIChatClient(credential=credential or AzureCliCredential())

    agent = chat_client.as_agent(
        name='AptPolicyAgent',
        instructions=(
            '너는 한국 아파트 정책/정세 분석 어시스턴트다. '
            '답변 전 반드시 answer_from_local_pdfs 도구를 호출해서 근거를 먼저 확인해라. '
            '근거가 부족하면 추측하지 말고 부족하다고 명시해라. '
            '답변은 한국어로 간결하게 작성하고, 근거는 파일명/페이지를 함께 제시해라.'
        ),
        tools=[tool.answer_from_local_pdfs, tool.load_local_pdfs],
    )

    return agent

agent = create_apt_policy_agent()
thread = agent.get_new_thread()
print('✅ AptPolicyAgent 생성 완료')


✅ AptPolicyAgent 생성 완료


In [11]:
# 실행 예시
import asyncio

query = '부동산 정책 관련해서 대출/세금/공급 이슈를 요약해줘'

async def _run_demo():
    result = await agent.run(query, thread=thread)
    print(result.text)

try:
    _loop = asyncio.get_running_loop()
except RuntimeError:
    _loop = None

if _loop and _loop.is_running():
    _loop.create_task(_run_demo())
else:
    asyncio.run(_run_demo())


대출, 세금, 공급 관련 부동산 정책 주요 이슈는 다음과 같습니다:

1. **대출**:  
   - 주택가격 수준에 따라 주택담보대출(LTV) 여신한도 차등화 및 전세대출에 대한 DSR 적용 등 대출 규제가 강화될 예정입니다.  
   - 생애최초 주택 구입자의 경우 LTV 최대 70%를 허용하며, 일정 기간 내 전입 의무를 부여합니다.  
   - 전세대출에 대해서는 보증비율을 강화하고, 다주택자의 대출을 금지하는 정책이 포함됩니다.
   - [근거: real_estate_policy_info.pdf, p.20/p.10]

2. **세금**:  
   - 초고가 주택의 취득거래 및 증여거래에 대한 전수 검증과 탈세행위에 대한 집중 신고 및 대응이 이루어지고 있습니다.
   - 부동산 세제 합리화 방안을 마련하기 위한 연구를 추진 중입니다.
   - [근거: real_estate_news.pdf, p.3 / real_estate_policy_info.pdf, p.20]

3. **공급**:  
   - 투기수요 및 거래질서를 교란하는 행위를 근절하기 위해 허위 거래 신고, 가격띄우기 등의 불법행위에 대한 기획조사와 엄정한 조치가 시행 중입니다.  
   - 거래질서 확립을 위해 범정부적인 대응체계와 감독기구 설치도 추진되고 있습니다.
   - [근거: real_estate_policy_info.pdf, p.17/p.18]

이와 같은 정책들은 부동산 시장의 안정화를 목표로 하고 있습니다.
